# Online Surrogate Model Dashboard Demo : Server
This is the first of two notebooks in the demo. Complete this notebook before moving on to Dashboard.ipynb.

## Project description 

In this demo, we will launch a PVAccess server and use its process variables as the inputs and outputs of an online surrogate model. We will then create an application using sliders to control the input process variables, and a number of data views to capture the ouput variables.

<br>
**This would be a nice place to add some specific info about the actual modeling, descriptions of inputs/outputs etc.**
<br>

This notebook is used to set up the PVAccess server. The server is responsible for running callbacks on updated inputs in order to generate model output, and serves the output process variables by PVAccess. This server can also be started from the command line using the command `python bin/cli.py serve start-server pva`.


## Description of model requirements 
There are several requirements necessary for running a model online with the server. 

- All variables, (input & output) must have unique names
- The output of the model must be configured to run a `predict` method that returns a dictionary mapping process variables to float outputs for single values, and a numpy array for image values. 
- The image model must contain the static method `prepare_image_from_pv` which contains a post-processing pipeline for the images after collected from the output process variables and before they are displayed in the dashboard. The output of this method must be in the form:

               {
                "image": [np.ndarray],
                "x": [float/int], 
                "y": [float/int],
                "dw": [float],
                "dh": [float],
            }


A base class (online_model.model.surrogate_model.SurrogateModel) is provided as an optional base class for model class development. Use of the base class is not necessary; however, it's abstract methods are intended to serve as template for the necessary methods required for server and application compatibility.


In [ ]:
# MAKE SURE THAT THE REPOSITORY ROOT IS IN THE PYTHONPATH
import sys
import os

# get absolute path of two parents up (brings us to project root)
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))

if module_path not in sys.path:
    sys.path.append(module_path)

# Package initialization
In order to simplify the code and reduce redundancy, all model specific configuration parameters are built and stored immediately at runtime. This initialization is also where hard-coded items like the model path are stored. These hard-coded items, which aren't derived at runtime will be moved to a designated configuration file.
<br>
### Hard-coded values
- MODEL_FILE
- STOCK_LASER_IMAGE
- STOCK_INPUT_SCALARS
- DEFAULT_INPUTS_SCALARS
- PREFIX
- REDUNDANT_INPUT_OUTPUT (used for patching current redundant input/ouptut)
- EXCLUDE_SLIDERS (sliders to exclude, currently input extents)
- MODEL_KWARGS
- ARRAY_PVS (used to indicate which variables are arrays)

### Built values (built from model info found in file at `MODEL_FILE` path)
- DEFAULT_INPUTS (composition of DEFAULT_LASER_IMAGE and DEFAULT_INPUTS_SCALARS)
- DEFAULT_LASER_IMAGE (loads STOCK_LASER_IMAGE)
- MODEL_INFO
- CMD_PVDB
- SIM_PVDB
<br>
As of now, introducing a new model will require manipulating the `online_model/__init__.py` file for the hard-coded values.

## Setting up the PVAccess server

The PVAccess server (and Channel Access server) require the input pvdb, output pvdb, model class, model kwarg configs, and prefix in order to start. 

In [ ]:
from online_model.server import pva
from online_model.model.MySurrogateModel import MySurrogateModel
from online_model import CMD_PVDB, SIM_PVDB, MODEL_KWARGS, PREFIX

In [ ]:
pva_server = pva.PVAServer(MySurrogateModel, MODEL_KWARGS, CMD_PVDB, SIM_PVDB, PREFIX)

In [ ]:
pva_server.start_server()

The server is now running. To continue the demo, move to the Dashboard.ipynb notebook (without shutting this tab down).